In [1]:
import os
import pickle
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import CrossEncoder

from utils.llmclass import RAGGenerator
from utils.search import HybridSearch
# Configuration
DB_PATH = os.path.join(os.getcwd(), "seamanuals")  # Safe path joining
BM25_PATH = os.path.join(DB_PATH, "bm25_retriever.pkl")

In [2]:
client = chromadb.PersistentClient(path=DB_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = client.get_or_create_collection(name="Sea-Database",
                                             embedding_function=embedding_func)
with open(BM25_PATH, "rb") as f:
    bm25_retriever = pickle.load(f)

In [3]:
search = HybridSearch.get(collection, bm25_retriever)

In [4]:
query = "What are the specific carpet area requirements for classrooms in a Maritime Training Institute based on student intake capacity?"
docs = search(query)

print("--- Retrieved Context ---")
for doc in docs:
    print(doc['rank'], doc['source'], doc['content'][:60])
    # print(f"Content: \n{doc['content']}")

--- Retrieved Context ---
1 training-manual-vol-i-pre-sea.pdf 2.9. Faculty room
2.9.1. A separate room not less than 8 m2 
2 training-manual-vol-i-pre-sea.pdf Directorate for approval if in compliance with the requireme
3 training-manual-vol-i-pre-sea.pdf and girls.
2.17. Hostel and residential facilities
The long 
4 training-manual-vol-i-pre-sea.pdf 1.4 Fee
The fee for seeking In-Principle approval, approval 
5 training-manual-vol-i-pre-sea.pdf The Institute shall have adequate original with copyright au
6 training-manual-vol-i-pre-sea.pdf 40 candidates respectively. Institutes approved prior to 1st
7 training-manual-vol-i-pre-sea.pdf PART 1 - APPROVAL OF PRE-SEA MARITIME TRAINING
INSTITUTES AN
8 training-manual-vol-i-pre-sea.pdf courses conducted by MTI....................................
9 training-manual-vol-i-pre-sea.pdf 2 1.7 Approval of courses conducted by MTI..................
10 training-manual-vol-i-pre-sea.pdf 1.7 Approval of courses conducted by MTI....................


In [5]:
rag = RAGGenerator(model_name='llama-3.2-1b:free')

In [7]:
# 1. SIMULATE RETRIEVAL
query = "What is the procedure when battling fires abroad other boats?"
docs = search(query)
context = ["-> ".join([doc['source'], doc['content']]) for doc in docs]
# 2. GENERATE RESPONSE
print(f"Query: {query}\n")
answer = rag.generate_answer(query, context)
print(f"Response:\n{answer}")

Query: What is the procedure when battling fires abroad other boats?

Response:
According to the provided context, the procedure when battling fires aboard other boats is as follows:

1. Brief crewmembers on appropriate procedures.
2. Assign each crewmember specific duties.
3. While en route to the scene, establish communications with the distressed boat.
4. Approach the boat from upwind.
5. If no one is onboard, circle the boat (at a safe distance) searching for PIWs.
6. Advise all persons aboard the boat to move to a flame and smoke-free area, topside.
7. Attempt to determine the extent and source of the fire. If it is not obvious, ask the personnel aboard the distressed boat where the fire is located.
8. If the fire is beyond the crewmembers' fire fighting capabilities, evacuate the persons from the distressed boat and call for assistance.
9. Check the physical condition of the survivors. If medical treatment is required, proceed to the nearest location where medical help can be adm